**Carregamento e Exploração dos Dados**

Primeiro vamos carregar as bibliotecas e os arquivos para fazer a análise, transformar em DataFrame, para ser mais fácil de visualizar as colunas e linhas e de manipular, posteriormente devemos usar a função head() para verificar se os dados foram importados corretamente e para ter a primeira noção das variáveis e valores.

In [ ]:
# Carregando bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
#descomentar o codigo abaixo depois de um tempo sem rodar o codigo
#!wget https://raw.githubusercontent.com/aline-pires/aprendizado_de_maquina/main/train.csv

dados = pd.read_csv('train.csv')
df = pd.DataFrame(dados)

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Depois de visualizar as 5 primeiras linhas e entender as variáveis. É de suma importância utilizar a função info() para saber qual é o tipo da variável (se é número inteiro, decimal e etc.), quantos valores não nulos temos em cada coluna e quantas variáveis nós temos

In [ ]:
# Tipo de dados de cada coluna
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


Também é muito importante fazer um resumo das variáveis, a função describe() calcula as principais medidas estatísticas, isso já uma ideia da distribuição e variância das variáveis.

In [ ]:
# Resumo das variáveis
print(df.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


Agora vamos ver qual é a proporção de sobreviventes:

In [ ]:
# Proporção de sobreviventes
print(df['Survived'].value_counts()/len(df))

Survived
0    0.616162
1    0.383838
Name: count, dtype: float64


Vemos que apenas 38% das pessoas que estavam no titanic sobreviveram e 62% faleceram.

**Pré processamento dos Dados**

Primeiro usamos a função isnull() para ver o número de informação faltante em cada variável.

In [ ]:
# Número de valores nulos por coluna
print(df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


A função mostrou que temos muitos valores ausentes na variável "Cabin", quase 80%, então já é um cenário mais crítico porque é muita informação faltante. Já a variável "Age" tem cerca de 20% de valores faltantes e a "Embarked" tem apenas 2 valores nulos, que não é tão ruim, pois podemos fazer uma técnica de imputação nesses casos. As demais colunas não contém NA, o que é bom cenário.
Para fazer a imputação de dados em "Age" utilizaremos a mediana, que é menos influenciável por outliers e em "Embarked" a moda, que é bom quando tem poucos NAs.

In [ ]:
# Substituindo NA's na coluna Age pela sua mediana
df['Age'].fillna(df['Age'].median(), inplace=True)

# Substituindo NA's na coluna Embarked pela sua moda
moda_emb = df['Embarked'].mode()[0]
df['Embarked'].fillna(moda_emb, inplace=True)

# Substituindo NA's na coluna Cabin por 'Desconhecido'
df['Cabin'].fillna('Desconhecido', inplace=True)

#verificando se imputou
print(df.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


/tmp/ipython-input-20-3874963724.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
/tmp/ipython-input-20-3874963724.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

Depois de fazer a imputação dos dados faltantes, vamos criar variáveis dummys para as variáveis categoricas ("Sex", "Embarked", "Pclass"), isso é importante para evitar multicolinearidade no modelo, que é basicamente ter informação repetida que afeta a independência dos dados, se temos apenas 2 respostas possíveis "homem" ou "mulher", fixamos apenas uma delas para entrar no modelo, porque se não é homem, com certeza é mulher, não precisa ter as duas informações.

In [ ]:
# Criando variáveis dummy a partir de Sex e Embarked
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Pclass'], drop_first=True)
# pclass transformada em dummy para que o "efeito" de cada classe na sobrevivência seja independente das outras classes
print(df.head())

   PassengerId  Survived                                               Name  \
0            1         0                            Braund, Mr. Owen Harris   
1            2         1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3         1                             Heikkinen, Miss. Laina   
3            4         1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5         0                           Allen, Mr. William Henry   

    Age  SibSp  Parch            Ticket     Fare         Cabin  Sex_male  \
0  22.0      1      0         A/5 21171   7.2500  Desconhecido      True   
1  38.0      1      0          PC 17599  71.2833           C85     False   
2  26.0      0      0  STON/O2. 3101282   7.9250  Desconhecido     False   
3  35.0      1      0            113803  53.1000          C123     False   
4  35.0      0      0            373450   8.0500  Desconhecido      True   

   Embarked_Q  Embarked_S  Pclass_2  Pclass_3  
0       False       

**Ajustando o Modelo de Regressão Logística**

O primeiro passo para ajustar o modelo é selecionar as variáveis preditoras e a variável resposta, depois vamos separar os dados em dois grupos: teste e treino, isso para as variáveis preditoras e para a resposta. Escolhemos 20% dos dados para ser o conjunto de teste e 80% para treino (a informação que está no teste não está no treino). Dessa forma, podemos rodar o modelo de regressão logística.

In [ ]:
# Selecionando variáveis
features = ['Pclass_2', 'Pclass_3', 'Age', 'SibSp', 'Parch', 'Fare',
            'Sex_male', 'Embarked_Q', 'Embarked_S']  # ajuste se tiver mais dummies

# Selecionando variáveis preditoras e reposta
X = df[features]
y = df['Survived']

In [ ]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2004)

# Ajuste do modelo de regressão logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

Após treinar o modelo, vamos testar com o conjunto de teste que separamos e verificar as medidas de desempenho, como a acurácia. Também é importante olhar a matriz de confusão, que mostra a quantidade de verdadeiro positivo e negativo, falso positivo e negativo. Além disso, fazer o relatório de classificação, que mostra de fato o desempenho do modelo.

In [ ]:
# Previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))
print("Relatório de classificação:\n", classification_report(y_test, y_pred))
# Precision: % de pessoas que o modelo disse que sobreviveriam e de fato sobreviveram
# Recall : % de pessoas que de fato sobreviveram e o modelo conseguiu identificar
# F1-score: média harmônica entre precisão e recall
# Support: o número de amostras reais de cada classe no conjunto de teste
# Macro avg: média simples das métricas entre as classes
# Weighted avg: média ponderada das métricas, considerando o número (support) em cada classe

Acurácia: 0.7988826815642458
Matriz de confusão:
 [[107  14]
 [ 22  36]]
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.83      0.88      0.86       121
           1       0.72      0.62      0.67        58

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.79      0.80      0.79       179



Calcularemos os coeficientes estimados de cada variável para ver quais são significativos para o modelo e a razão de chance, pois a regressão logística modela o logaritmo das chances (log-odds) de o evento ocorrer, essa medida indica o quanto a chance do evento multiplica para cada aumento unitário da variável.

In [ ]:
# Coeficientes e odds ratio do modelo
coef = model.coef_[0]
odds_ratios = np.exp(coef)

coef_df = pd.DataFrame({
    'Variável': features,
    'Coeficiente': coef,
    'Odds Ratio': odds_ratios
}).sort_values(by='Odds Ratio', ascending=False)

print(coef_df)

# Cada coeficiente representa o quanto o log-odds muda para um aumento unitário na variável, mantendo as outras variáveis fixas.

     Variável  Coeficiente  Odds Ratio
5        Fare     0.006698    1.006721
7  Embarked_Q    -0.019720    0.980473
2         Age    -0.037881    0.962827
4       Parch    -0.069754    0.932623
3       SibSp    -0.312338    0.731734
8  Embarked_S    -0.361794    0.696426
0    Pclass_2    -0.441578    0.643021
1    Pclass_3    -1.799280    0.165418
6    Sex_male    -2.452343    0.086092


Variáveis mais influentes (maior efeito na probabilidade de sobrevivência):


*   Sexo -> homens têm 91,4% menos chance de sobreviver do que mulheres, mantendo o resto constante, portanto, ser homem reduz fortemente a chance de sobreviver.
*   Classe -> Passageiros da 3ª classe têm chances bem menores de sobreviver comparados à 1ª classe (que é a referência), cerca de 83,5% menos. Passageiros da 2ª classe também reduz as chances comparados com à 1ª classe, mas é 35,7% menos.
*   Local de Embarque -> Passageiros que embarcaram em Southampton têm menos chances sobreviver comparados à aos que embarcaram em Cherbourg, cerca de 30,4% a menos.

As demais variáveis não têm tanto efeito na probabilidade de sobrevivência, mas a Fare tem um efeito positivo, mas pequeno (quanto maior a tarifa paga, ligeiramente maior a chance de sobreviver). Já as variáveis Age, Parch e SibSp têm efeitos negativos, mas bem pequenos.

**Qual a importância de tratar os valores ausentes e codificar as variáveis categóricas antes de ajustar o modelo?**

O modelo de regressão logística não aceita valores faltantes (NA), além disso, se não houver uma imputação de dados correta pode introduzir vieses. Sobre a categorização das variáveis é importante porque, além de evitar multicolinearidade, o próprio modelo só aceita valores numéricos na sua entrada.

**Quais são as limitações do modelo de regressão logística para este problema?**

O modelo assume que o logit da probabilidade de sobrevivência é uma função linear das variáveis, e se houver relações não são lineares nos dados, o modelo não irá capturar isso. Além disso, é um modelo bem simples, então não consegue capturar relações mais complexas e nem lida muito bem com dados desbalanceados, como é o nosso caso, temos muitas pessoas que não sobreviveram do que sobreviveram (tanto é que pela matriz de confunsão o modelo tem mais falso positivo). Para contornar esse problema os modelos mais complexos como árvore de descisão e redes neurais provavelmente iria ter um melhor desempenho.